# Data Frames – Integration with Hive and JDBC

Let us see how we can use Data Frames to get data from Hive and Relational databases using JDBC and then process data using operations available on Data Frames.

* Recap of Hive
* Reading and Writing Data – Hive tables
* Reading and Writing Data – JDBC
* Querying data from Hive tables
* Hive DDL and DML – Data Frame Operations
* Hive DDL and DML – Spark SQL
* Hive Functions – Overview

We will be using labs for the demonstration of interacting with Hive. It is straightforward to connect to Hive from Spark when both of them are integrated. If you want to explore Hive using Spark on your PC, I would recommend to use Cloudera’s QuickStart VM or Hortonworks Sandbox. Setting up Hive and integrating with Spark is a bit tedious (especially on Windows).

Data Frames – Integration with Hive and JDBC
JUNE 26, 2018 By Durga Gadiraju Leave a Comment

Topic Progress:               
← Back to Lesson
Let us see how we can use Data Frames to get data from Hive and Relational databases using JDBC and then process data using operations available on Data Frames.

Recap of Hive
Reading and Writing Data – Hive tables
Reading and Writing Data – JDBC
Querying data from Hive tables
Hive DDL and DML – Data Frame Operations
Hive DDL and DML – Spark SQL
Hive Functions – Overview
We will be using labs for the demonstration of interacting with Hive. It is straightforward to connect to Hive from Spark when both of them are integrated. If you want to explore Hive using Spark on your PC, I would recommend to use Cloudera’s QuickStart VM or Hortonworks Sandbox. Setting up Hive and integrating with Spark is a bit tedious (especially on Windows).


### Recap of Hive

Let us see a quick recap of Hive.

* Hive is used to create databases, tables and process data in Big Data Clusters.
* It uses HDFS for the file system. A Hive database or a table or a partition is nothing but a directory in HDFS.
* Once we create table metadata will be stored in an RDBMS database configured. We have configured our Hive with MySQL.
* Hive also have query engine. A query will be typically compiled into Map Reduce Job.
* Once we have tables created, we can run queries using different query engines.
    * Hive
    * Tez
    * Impala
    * Spark SQL
    * Presto
    * and more
* Hive support DDL, batch data load as well as inserting query results on top of running SQL type queries.
* Let us create Hive Database, create few tables and process using standard Hive Queries before we jump into Spark SQL.

### Reading and Writing Data – Hive tables

Let us see how we can read and write data using Hive Tables leveraging Data Frame Operations.

* SparkSession have an API under read to get data directly from Hive Tables.
* We can use SparkSession’s (spark) read.table to read data from Hive table.
* Data from Hive Table will be loaded into Data Frame. We can now leverage Data Frame Operations to process the data.

In [7]:
import org.apache.spark.sql.functions._

In [2]:
spark.sql("use trainingdemo")

[]

In [3]:
spark.sql("show tables").show

+------------+--------------------+-----------+
|    database|           tableName|isTemporary|
+------------+--------------------+-----------+
|trainingdemo|       daily_revenue|      false|
|trainingdemo|order_count_by_st...|      false|
|trainingdemo|         order_items|      false|
|trainingdemo|       order_revenue|      false|
|trainingdemo|              orders|      false|
+------------+--------------------+-----------+



In [4]:
spark.read.table("trainingdemo.orders").show(false)

+--------+---------------------+-----------------+---------------+
|order_id|order_date           |order_customer_id|order_status   |
+--------+---------------------+-----------------+---------------+
|1       |2013-07-25 00:00:00.0|11599            |CLOSED         |
|2       |2013-07-25 00:00:00.0|256              |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00.0|12111            |COMPLETE       |
|4       |2013-07-25 00:00:00.0|8827             |CLOSED         |
|5       |2013-07-25 00:00:00.0|11318            |COMPLETE       |
|6       |2013-07-25 00:00:00.0|7130             |COMPLETE       |
|7       |2013-07-25 00:00:00.0|4530             |COMPLETE       |
|8       |2013-07-25 00:00:00.0|2911             |PROCESSING     |
|9       |2013-07-25 00:00:00.0|5657             |PENDING_PAYMENT|
|10      |2013-07-25 00:00:00.0|5648             |PENDING_PAYMENT|
|11      |2013-07-25 00:00:00.0|918              |PAYMENT_REVIEW |
|12      |2013-07-25 00:00:00.0|1837             |CLOSED      

In [5]:
val orders = spark.read.table("orders")
orders.
  groupBy("order_date").
  count.
  show

+--------------------+-----+
|          order_date|count|
+--------------------+-----+
|2013-08-13 00:00:...|  146|
|2013-10-12 00:00:...|  324|
|2013-11-15 00:00:...|  270|
|2014-03-19 00:00:...|  260|
|2014-04-26 00:00:...|  502|
|2013-09-16 00:00:...|  242|
|2013-09-20 00:00:...|  278|
|2013-12-31 00:00:...|  532|
|2013-09-06 00:00:...|  552|
|2014-06-15 00:00:...|  256|
|2013-12-24 00:00:...|  340|
|2014-01-07 00:00:...|  326|
|2014-06-07 00:00:...|  382|
|2013-10-14 00:00:...|  278|
|2013-11-11 00:00:...|  492|
|2014-01-27 00:00:...|  326|
|2014-01-29 00:00:...|  316|
|2014-02-14 00:00:...|  348|
|2014-04-15 00:00:...|  360|
|2014-04-22 00:00:...|  288|
+--------------------+-----+
only showing top 20 rows



orders = [order_id: int, order_date: string ... 2 more fields]


[order_id: int, order_date: string ... 2 more fields]

In [8]:
val orderItems = spark.read.table("order_items")
orderItems.
  groupBy("order_item_order_id").
  agg(round(sum("order_item_subtotal"), 2).alias("order_revenue")).
  show

+-------------------+-------------+
|order_item_order_id|order_revenue|
+-------------------+-------------+
|                148|       959.98|
|                463|      1659.84|
|                471|       339.96|
|                496|        883.9|
|               1088|       499.94|
|               1580|        599.9|
|               1591|       879.72|
|               1645|      3019.58|
|               2366|       599.94|
|               2659|      1449.82|
|               2866|      1139.92|
|               3175|       419.94|
|               3749|       287.94|
|               3794|        599.9|
|               3918|      1659.86|
|               3997|       1159.9|
|               4101|       259.98|
|               4519|       159.96|
|               4818|       799.96|
|               4900|       359.94|
+-------------------+-------------+
only showing top 20 rows



orderItems = [order_item_id: int, order_item_order_id: int ... 4 more fields]


[order_item_id: int, order_item_order_id: int ... 4 more fields]

* Processed Data Frame can be saved into a Hive table using multiple APIs under **spark.write**
    * **saveAsTable** – creates a new table in Hive with Parquet file format
    * **insertInto** – insert data frame into existing tables
We can use mode to append or overwrite into the table. Modes can be passed as string or org.apache.spark.sql.SaveMode object.

In [10]:
spark.sql("use trainingdemo")
spark.sql("show tables").show

+------------+--------------------+-----------+
|    database|           tableName|isTemporary|
+------------+--------------------+-----------+
|trainingdemo|       daily_revenue|      false|
|trainingdemo|order_count_by_st...|      false|
|trainingdemo|         order_items|      false|
|trainingdemo|       order_revenue|      false|
|trainingdemo|              orders|      false|
+------------+--------------------+-----------+



In [11]:
spark.read.table("orders").show(false)
spark.read.table("trainingdemo.orders").show(false)

+--------+---------------------+-----------------+---------------+
|order_id|order_date           |order_customer_id|order_status   |
+--------+---------------------+-----------------+---------------+
|1       |2013-07-25 00:00:00.0|11599            |CLOSED         |
|2       |2013-07-25 00:00:00.0|256              |PENDING_PAYMENT|
|3       |2013-07-25 00:00:00.0|12111            |COMPLETE       |
|4       |2013-07-25 00:00:00.0|8827             |CLOSED         |
|5       |2013-07-25 00:00:00.0|11318            |COMPLETE       |
|6       |2013-07-25 00:00:00.0|7130             |COMPLETE       |
|7       |2013-07-25 00:00:00.0|4530             |COMPLETE       |
|8       |2013-07-25 00:00:00.0|2911             |PROCESSING     |
|9       |2013-07-25 00:00:00.0|5657             |PENDING_PAYMENT|
|10      |2013-07-25 00:00:00.0|5648             |PENDING_PAYMENT|
|11      |2013-07-25 00:00:00.0|918              |PAYMENT_REVIEW |
|12      |2013-07-25 00:00:00.0|1837             |CLOSED      

In [12]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [13]:
val orders = spark.read.table("orders")
val orderCountByStatus = orders.
  groupBy("order_date").
  count

orders = [order_id: int, order_date: string ... 2 more fields]
orderCountByStatus = [order_date: string, count: bigint]


[order_date: string, count: bigint]

In [15]:
// Dosent have access.

// orderCountByStatus.
//   write.
//   mode("overwrite").
//   saveAsTable("order_count_by_status")

lastException: Throwable = null


Name: Syntax Error.
Message: 
StackTrace: 

In [26]:
import org.apache.spark.sql.SaveMode

// orderCountByStatus.
//   write.
//   mode(SaveMode.Append).
//   saveAsTable("order_count_by_status")

In [17]:
val orderItems = spark.read.table("order_items")
val orderRevenue = orderItems.
  groupBy("order_item_order_id").
  agg(round(sum("order_item_subtotal"), 2).alias("order_revenue"))

orderItems = [order_item_id: int, order_item_order_id: int ... 4 more fields]
orderRevenue = [order_item_order_id: int, order_revenue: double]


lastException: Throwable = null


[order_item_order_id: int, order_revenue: double]

In [1]:
spark.sql(s"""CREATE TABLE IF NOT EXISTS trainingdemo.order_revenue (
              order_id INT, 
              revenue FLOAT)""")


[]

In [27]:
orderRevenue.
  write.
  mode(SaveMode.Overwrite).
  insertInto("order_revenue")

### Reading and Writing Data – JDBC

Let us see how we can read data from a remote database using Spark JDBC.

* To connect to a Database, we need
    * JDBC Jar File, which can be passed using –packages or –jars or as part of the fat jar file
    * Server ip or DNS alias on which Database is running
    * Credentials for authentication and authorization to the Database
    * Database and Table Names we want to read the data from.
* We can read data from the remote database into Data Frame using **spark.read.jdbc**
* Database Connectivity information can be passed as options.
* We can also use **spark.read.format(“jdbc”)** along with options.
* Once data is processed we can write data back to the remote database using **write.jdbc** or **write.format**
* We can either append to an existing table or overwrite it, but we will not be able to update or merge table over JDBC.
* We can see the list of options in official documentation. Spark JDBC can replace Sqoop to get data from Relational Databases into HDFS and vice-versa.

### Querying Data from Hive Tables

Apart from reading data from Hive Tables using Data Frame APIs, we can also use spark.sql to read data from Hive Tables as well as to write data to Hive Tables.

* spark.sql can be used to issue any valid Hive Command or Query
* It will always return a Data Frame
* We need to use **show** to preview the data or collect to print the results
* If we issue a complex query, that query will be executed and results will be returned to Data Frame.
* Once Data Frame is created we can use all Data Fram Operations.
* Let us run **select * from orders** using spark.sql and preview the results.
* We can run any valid query using spark.sql. We will see more examples on Spark SQL in upcoming sections.

### Hive DDL and DML – Spark

Let us see how we can perform DDL and DML on top of Hive tables using Spark.

* We can use Data Frame Operations or Spark SQL to perform Hive DDL as well as DML.
* For Data Frame Operations we can use **write.saveAsTable** or **write.insertInto**. We can also use mode to overwrite an existing table or append to it. Options can be used to define additional properties while saving into Hive tables.
* We can pick the queries which are executed to create the tables and run them using **spark.sql**. We need to format them as strings.

### Hive Functions – Overview

Let us go through the details about Hive Functions. For most of the functions we have seen as part of org.apache.spark.sql.functions, we have counterparts in Hive.

* We can get the list of functions by using show functions command.
* We can get basic usage and syntax of a given function by using **describe function** command.
* As mentioned earlier we can run these commands as part of **spark.sql**
* Make sure to use **show(false)** to get the complete details.
* We can categorize the functions into below groups.
    * String Manipulation Functions
        * trim, rtrim, ltrim
        * lpad, rpad
        * length
        * substring, split
    * Date Manipulation Functions
        * date_add, date_sub, datediff
        * date_format
        * trunc
     * Type Conversion Functions
        * cast as
     * CASE WHEN
     * and more
* Syntax can be different between functions in the package **org.apache.spark.sql.functions** and functions made available via Hive.     